In [1]:
import shutil
import os
import numpy as np
from core.edfhelper import Patient
from core.datahelper import DataSplitter, DataAE

In [2]:
def preprocess_data_patient(p, out_dir):
    signals, _, ssignals, _ = p.get_chunks_5s()
    dir_path = os.path.join(out_dir, str(p.patient_id))
    
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
    os.mkdir(dir_path)
    os.mkdir(os.path.join(dir_path, 'signals_0'))
    os.mkdir(os.path.join(dir_path, 'signals_1'))

    for indx, s in enumerate(signals):
        np.save(os.path.join(dir_path, 'signals_0', 'signals_{0}_{1}.npy'.format(0, indx)), s)
    for indx, seizure in enumerate(ssignals):
        s_path = os.path.join(dir_path, 'signals_1', str(indx))
        os.mkdir(s_path)
        for c_indx, chunk in enumerate(seizure):
            np.save(os.path.join(s_path, 'seizure_{0}'.format(c_indx)), chunk)

def preprocess_data(patients_ids, in_dir, out_dir):
    for i in patients_ids:
        indx = i
        if indx < 10:
            indx = '0' + str(indx)
        else:
            indx = str(indx)
        p = Patient(os.path.join(in_dir, f'chb{indx}'), i)
        preprocess_data_patient(p, out_dir)

In [3]:
patients_ids = [1, 3, 7, 9, 10, 20, 21, 22]
patients_dir_in = 'D:\\CHBData/physionet.org/files/chbmit/1.0.0'
patients_dir_out = 'C:\CHBData\preprocess'

In [ ]:
#preprocess all data
preprocess_data(patients_ids, patients_dir_in, patients_dir_out)

In [ ]:
def generate_ae_files(path_to_data, path_to_split):
    train_files, test_files = DataAE.get_files(path_to_data)
    DataAE.save_files(path_to_split, train_files, test_files)

def generate_p_s_files(path_to_data, p, s):
    train, test = DataSplitter.split_data_p(path_to_data, p, s)
    DataSplitter.save_to_file(path_to_data, p, s, train, test)

def generate_files_for_all_patients(path_to_data):
    for p in os.listdir(path_to_data):
        if not os.path.isdir(os.path.join(path_to_data, p)):
            continue
        for s in range(DataSplitter.get_seizures_number(path_to_data, p)):
            generate_p_s_files(path_to_data, p, s)

In [ ]:
#generate files contains filenames to use in train / test for each patient 
generate_files_for_all_patients(patients_dir_out)

In [ ]:
#generate files contains filenames to use in train / test for autoencoder
generate_ae_files(patients_dir_out, patients_dir_out)